In [17]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
from tensorflow.keras.layers import Dense, Flatten, Reshape, Dropout, Conv1D, Conv2D, MaxPooling1D, BatchNormalization, Input
from tensorflow.keras.utils import to_categorical


In [18]:
from KeplerModel import KeplerModel

In [22]:
""" 
Example of an artificial neural network model
Derived from class KerasModel
"""
class CNNModel(KeplerModel):
  def __init__(self, modelname, hparam):
    super().__init__(modelname, hparam)

    # Construct the model
    inputShape = (2,256)
    inputs = Input(shape=inputShape)
    self.BuildModel(inputs, hparam)

    self.SetLearningRate(hparam['learning_rate'])
    self.SetLoss('mae')
    self.SetMetrics(['mae','mse'])
    self.SetOptimizer(Adam)
    #self.Compile()
  
    self.SetModelName("cnn")
    self.SetModelVersion(1)
    self.SetModelTrial(1)
    
    
  def CNN(self, inputs, hp):
    # Construct an MLP model

    m = inputs
    
    self.dropout=np.full((hp['layers']),hp['dropout'])
    batchNorm = hp['batch_norm']
    act = hp['activation']
    units = hp['units']
    layers = hp['layers']
    expansion = hp['expansion']
    kernel = hp['kernel']

    for layer in range(layers):
      m = Conv1D(units, kernel, padding='same')(m)
      if batchNorm:
        m = BatchNormalization()(m)
      m = activations.get(act)(m)
      #m = MaxPooling1D(pool_size=2)(m)
      if not batchNorm and self.dropout[layer] > 0.0:
        m = Dropout(self.dropout[layer])(m)
      units *= expansion

    m = Flatten()(m)
    m = Dense(hp['final_units'])(m)
    m = activations.get(hp['final_activation'])(m)
    if batchNorm:
      m = BatchNormalization()(m)
    if not batchNorm and hp['final_dropout'] > 0.0:
      m = Dropout(hp['final_dropout'])(m)
    m = Dense(5, activation=hp['final_activation'])(m)

    target = Dense(1, activation=hp['target_activation'], name='target')(m)
    return target
    
  def BuildModel(self, inputs, hp):
    # Construct the model

    target = self.CNN(inputs, hp)
    m = Model(inputs=inputs, outputs=[target], name="cnn")
    self.SetModel(m)
    return m
    

In [23]:
def main():
  hyperParam = {
    'version': 1,
    'revision': 1,
    'trial': 1,
    'units': 8,
    'expansion': 2,
    'layers': 5,
    'kernel': 2,
    'activation': 'relu',
    'batch_norm': True,
    'dropout': 0.0,
    'final_units': 4096,
    'final_dropout': 0.0,
    'final_activation': 'relu',
    'target_activation': 'relu',
    'learning_rate': 0.001,
    'batch_size': 64,
    'epoch': 0,
    'begin': 0,
    'epochs': 10,
    'patience': 0,
    'threshold': 0,
  }

  m = CNNModel('cnn', hyperParam)
  print(m.model.summary())

  m.SetModelTrial(1)
  m.SetModelRevision(1)
  m.SetModelPath("./model")
  m.SaveModel()

In [24]:
# -
if __name__ == '__main__':
  main()


Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 2, 256)]          0         
                                                                 
 conv1d_5 (Conv1D)           (None, 2, 8)              4104      
                                                                 
 batch_normalization_1 (Bat  (None, 2, 8)              32        
 chNormalization)                                                
                                                                 
 tf.nn.relu_1 (TFOpLambda)   (None, 2, 8)              0         
                                                                 
 conv1d_6 (Conv1D)           (None, 2, 16)             272       
                                                                 
 batch_normalization_2 (Bat  (None, 2, 16)             64        
 chNormalization)                                              

INFO:tensorflow:Assets written to: model/cnnv1r1t1-e0/assets


In [25]:
!ls -lt model

total 0
drwxr-xr-x 4 root root 106 Oct  6 08:29 cnnv1r1t1-e0
drwxr-xr-x 4 root root 106 Oct  6 08:09 mlpv1r2t1-e10
drwxr-xr-x 4 root root 106 Oct  6 08:08 mlpv1r2t1-e9
drwxr-xr-x 4 root root 106 Oct  6 08:06 mlpv1r2t1-e7
drwxr-xr-x 4 root root 106 Oct  6 08:05 mlpv1r2t1-e6
drwxr-xr-x 4 root root 106 Oct  6 08:04 mlpv1r2t1-e5
drwxr-xr-x 4 root root 106 Oct  6 08:03 mlpv1r2t1-e4
drwxr-xr-x 4 root root 106 Oct  6 08:02 mlpv1r2t1-e3
drwxr-xr-x 4 root root 106 Oct  6 08:01 mlpv1r2t1-e2
drwxr-xr-x 4 root root 106 Oct  6 08:00 mlpv1r2t1-e1
drwxr-xr-x 4 root root 106 Oct  6 07:57 mlpv1r2t1-e0
drwxr-xr-x 4 root root 106 Oct  6 07:57 mlpv1r1t1-e0
drwxr-xr-x 4 root root 106 Oct  6 06:15 mlpv1r1t1-e10
drwxr-xr-x 4 root root 106 Oct  6 06:14 mlpv1r1t1-e9
drwxr-xr-x 4 root root 106 Oct  6 06:13 mlpv1r1t1-e8
drwxr-xr-x 4 root root 106 Oct  6 06:12 mlpv1r1t1-e7
drwxr-xr-x 4 root root 106 Oct  6 06:11 mlpv1r1t1-e6
drwxr-xr-x 4 root root 106 Oct  6 06:10 mlpv1r1t1-e5
drwxr-xr-x 4 root root 106 Oct  6 06